## Playing around with dataset, no useful DNN in this code

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import seaborn as sb
import matplotlib as mpl
import h5py
import pandas as pd 
import tables
from pandas.compat import StringIO, BytesIO 

mpl.rcParams['figure.dpi']= 250
pd.__file__

from torch.utils import data
#from hdf5dataset import HDF5Dataset

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Import hdf5 file and look at what is inside

In [ ]:
#open hdf5 file for reading
    #hdf_buggy = pd.HDFStore('/data/wfedorko/top_tagging_spr1/subsampled/gen_spr1_full_zpdijet_cms_50pileup_notrim/flat_distribution2.hdf',mode='r') #can't access data using this - use h5py.File instead
    #hdf_buggy.keys() #this is an empty array
hdf_bug = h5py.File('/data/wfedorko/top_tagging_spr1/subsampled/gen_spr1_full_zpdijet_cms_50pileup_notrim/flat_distribution2.hdf',mode='r')

print(list(hdf_bug.keys()))

data = hdf_bug['data']
print(data.shape)
print(type(data))
print(data[0][0])
print(data[-1][0])

sequence_len = hdf_bug['sequence lengths']
print(sequence_len.shape)
print(sequence_len[-1])

#table_buggy = hdf_buggy.get('/table')

#hdf_val = pd.HDFStore('val.h5',mode='r')
#hdf_val.keys()
#table_val = hdf.get('/table')

#hdf_test = pd.HDFStore('test.h5',mode='r')
#hdf_test.keys()
#table_test = hdf.get('/table')


# Trying to figure out what the stucture of the data set is and how many features are in each jet

In [105]:
print(data[1].size)
#print(list(data[1]))

#print(data[1][0:17].size)
print(data[1][0:3])
#print(data[1][14])
#print(data[1][20:27])

print(sequence_len[1])


2816
[  0.        1.      185.94533]
181.0


In [ ]:
#still guessing...
print('the number of features for constituent might be close to',2816/205)

#max(sequence_len) is taking WAY too long, so here's a quick way to see what range of numbers sequence lengths contains
for i in range(111150,111175):
    print(sequence_len[i])

In [ ]:
#create pytorch tensor
train_tensor = torch.tensor(data)
#val_tensor = torch.tensor(table_val.values)
#test_tensor = torch.tensor(table_test.values)


#len(torch_tensor) 1211000

# another option, but this appears to be the same as using table...
# d = pd.read_hdf('train.h5','table')
# torch_tensor = torch.tensor(d.values)

# Not sure what this machine learning package below is, but I have kept it because it looks like it has some useful CNN architecture 

In [ ]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001


train_loader = torch.utils.data.DataLoader(dataset=train_tensor,
                                           batch_size=batch_size, 
                                           shuffle=True)

sub_train_loader = torch.utils.data.DataLoader(dataset=sub_train_tensor,
                                           batch_size=batch_size, 
                                           shuffle=True)

In [85]:
class NetBatchNorm(nn.Module):
    
    # Constructor
    def __init__(self, in_size, n_hidden1, n_hidden2, out_size):
        super(NetBatchNorm, self).__init__()
        self.linear1 = nn.Linear(in_size, n_hidden1)
        self.linear2 = nn.Linear(n_hidden1, n_hidden2)
        self.linear3 = nn.Linear(n_hidden2, out_size)
        self.bn1 = nn.BatchNorm1d(n_hidden1)
        self.bn2 = nn.BatchNorm1d(n_hidden2)
        
    # Prediction
    def forward(self, x):
        x = self.bn1(torch.sigmoid(self.linear1(x)))
        x = self.bn2(torch.sigmoid(self.linear2(x)))
        x = self.linear3(x)
        return x
    
    # Activations, to analyze results 
    def activation(self, x):
        out = []
        z1 = self.bn1(self.linear1(x))
        out.append(z1.detach().numpy().reshape(-1))
        a1 = torch.sigmoid(z1)
        out.append(a1.detach().numpy().reshape(-1).reshape(-1))
        z2 = self.bn2(self.linear2(a1))
        out.append(z2.detach().numpy().reshape(-1))
        a2 = torch.sigmoid(z2)
        out.append(a2.detach().numpy().reshape(-1))
        return out
    
class Net(nn.Module):
    
    # Constructor
    def __init__(self, in_size, n_hidden1, n_hidden2, out_size):

        super(Net, self).__init__()
        self.linear1 = nn.Linear(in_size, n_hidden1)
        self.linear2 = nn.Linear(n_hidden1, n_hidden2)
        self.linear3 = nn.Linear(n_hidden2, out_size)
    
    # Prediction
    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = self.linear3(x)
        return x
    
    # Activations, to analyze results 
    def activation(self, x):
        out = []
        z1 = self.linear1(x)
        out.append(z1.detach().numpy().reshape(-1))
        a1 = torch.sigmoid(z1)
        out.append(a1.detach().numpy().reshape(-1).reshape(-1))
        z2 = self.linear2(a1)
        out.append(z2.detach().numpy().reshape(-1))
        a2 = torch.sigmoid(z2)
        out.append(a2.detach().numpy().reshape(-1))
        return out 
    
    
def train(model, criterion, train_loader, validation_loader, optimizer, epochs=100):
    i = 0
    useful_stuff = {'training_loss':[], 'validation_accuracy':[]}  

    for epoch in range(epochs):
        for i, (x, y) in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            z = model(x.view(-1, 28 * 28))
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            useful_stuff['training_loss'].append(loss.data.item())
            
        correct = 0
        for x, y in validation_loader:
            model.eval()
            yhat = model(x.view(-1, 28 * 28))
            _, label = torch.max(yhat, 1)
            correct += (label == y).sum().item()
            
        accuracy = 100 * (correct / len(validation_dataset))
        useful_stuff['validation_accuracy'].append(accuracy)
    
    return useful_stuff




In [101]:
# Create Data Loader for both train and validating

train_loader = torch.utils.data.DataLoader(dataset=train_tensor, batch_size=2000, shuffle=True)
#validation_loader = torch.utils.data.DataLoader(dataset=val_tensor, batch_size=5000, shuffle=False)


sub_train_loader = torch.utils.data.DataLoader(dataset=sub_train_tensor,
                                           batch_size=2000, 
                                           shuffle=True)

type(train_loader)

torch.utils.data.dataloader.DataLoader

In [99]:
# Create the criterion function
criterion = nn.CrossEntropyLoss

# Set the parameters
input_dim = 12
hidden_dim = 100
output_dim = 10

In [79]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out